***GENERATED CODE FOR bankdepositanalysis PIPELINE***
***DON'T EDIT THIS CODE***

**CONNECTOR FUNCTIONS TO READ DATA FROM DATABRICKS FILESYSTEM**

In [ ]:
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class DBFSConnector:

    def fetch(spark, config):
        df = spark.read.\
            options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                    inferschema='true',
                    delimiter=eval(config)["delimiter"])\
            .csv(eval(config)['url'])
        display(df.limit(2).toPandas())
        return df

    def put(df, spark, config):
        return df.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                              delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


**TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA**

In [ ]:
import json
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import col, when
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def replaceByMean(self, feature, df, mean_=-1):

        meanValue = df.select(mean(col(feature.name)).alias(
            'mean')).collect()[0]["mean"]
        df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                                         meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        maxValue = df.select(max(col(feature.name)).alias('max')).collect()[
            0]["max"]
        df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        minValue = df.select(min(col(feature.name)).alias('min')).collect()[
            0]["min"]
        df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        stddevValue = df.select(stddev(col(feature.name)).alias(
            'stddev')).collect()[0]["stddev"]
        df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        fillValue = df.where(col(feature.name).isNotNull()
                             ).head(1)[0][feature.name]
        df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def StringIndexerTransform(df, params):
    dfReturn = df
    feature = params["feature"]

    dfReturn = dfReturn.fillna({feature: ''})
    outcol = feature + "_transform"
    indexer = StringIndexer(
        inputCol=feature, outputCol=outcol, handleInvalid="skip")
    indexed = indexer.fit(dfReturn).transform(dfReturn)
    indexed = indexed.drop(feature).withColumnRenamed(outcol, feature)
    dfReturn = indexed
    distinct_values_list = dfReturn.select(
        feature).distinct().rdd.map(lambda r: r[0]).collect()
    len_distinct_values_list = len(distinct_values_list)
    if len_distinct_values_list <= 4:
        changed_type_df = dfReturn.withColumn(
            feature, dfReturn[feature].cast(IntegerType()))
        return changed_type_df
    # changed_type_df.show(3)
    return dfReturn


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': {'feature_label': 'job'}, 'feature': 'job', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
                                                  'count': '5647', 'mean': '', 'stddev': '', 'min': 'admin.', 'max': 'unknown', 'missing': '0'}, 'transformation': 'String Indexer'})
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': {'feature_label': 'marital'}, 'feature': 'marital', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '5647', 'mean': '', 'stddev': '', 'min': 'divorced', 'max': 'single', 'missing': '0'}, 'transformation': 'String Indexer'})
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': {'feature_label': 'education'}, 'feature': 'education', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '5647', 'mean': '', 'stddev': '', 'min': 'primary', 'max': 'unknown', 'missing': '0'}, 'transformation': 'String Indexer'})
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': {'feature_label': 'default'}, 'feature': 'default', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '5647', 'mean': '', 'stddev': '', 'min': 'no', 'max': 'yes', 'missing': '0'}, 'transformation': 'String Indexer'})
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': {'feature_label': 'housing'}, 'feature': 'housing', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '5647', 'mean': '', 'stddev': '', 'min': 'no', 'max': 'yes', 'missing': '0'}, 'transformation': 'String Indexer'})
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': {'feature_label': 'loan'}, 'feature': 'loan', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '5647', 'mean': '', 'stddev': '', 'min': 'no', 'max': 'yes', 'missing': '0'}, 'transformation': 'String Indexer'})
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': {'feature_label': 'contact'}, 'feature': 'contact', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '5647', 'mean': '', 'stddev': '', 'min': 'cellular', 'max': 'unknown', 'missing': '0'}, 'transformation': 'String Indexer'})
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': {'feature_label': 'month'}, 'feature': 'month', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '5647', 'mean': '', 'stddev': '', 'min': 'apr', 'max': 'sep', 'missing': '0'}, 'transformation': 'String Indexer'})
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': {'feature_label': 'poutcome'}, 'feature': 'poutcome', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '5647', 'mean': '', 'stddev': '', 'min': 'failure', 'max': 'unknown', 'missing': '0'}, 'transformation': 'String Indexer'})
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': {'feature_label': 'deposit'}, 'feature': 'deposit', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '5647', 'mean': '', 'stddev': '', 'min': 'no', 'max': 'yes', 'missing': '0'}, 'transformation': 'String Indexer'})
        display(transformationDF.limit(2).toPandas())
        return transformationDF


**AUTOML FUNCTIONS**

In [ ]:
from tpot import TPOTClassifier
from sklearn.model_selection import train_test_split
import pyspark


def functionClassification(sparkDF, listOfFeatures, label):
    sparkDF.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    df = sparkDF.toPandas()
    X = df.drop(label, axis=1).values
    y = df[label].values
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, random_state=1, test_size=0.1)
    tpotModel = TPOTClassifier(verbosity=3, n_jobs=-1, generations=10, max_time_mins=5,
                               population_size=15)
    tpotModel.fit(X_train, y_train)
    display(" Accuracy of Model : %s" % tpotModel.score(X_test, y_test))
    data = {'model': tpotModel,
            'X_test': X_test,
            'y_test': y_test,
            'label': label,
            'columnNames': sparkDF.columns}
    return data


**READING DATAFRAME**

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

%run bankdepositanalysisHooks.ipynb
try:
	sourcePreExecutionHook()

	bankdepositanalysisdbfs = DBFSConnector.fetch(spark, "{'url': '/Demo/BankDepositTrain.csv', 'file_type': 'Delimeted', 'delimiter': ',', 'is_header': 'Use Header Line'}")

	sourcePostExecutionHook(bankdepositanalysisdbfs)

except Exception as ex: 
	logging.error(ex)


**TRANSFORMING DATAFRAME**

In [ ]:
%run bankdepositanalysisHooks.ipynb
try:
	transformationPreExecutionHook()

	bankdepositanalysisautofe = TransformationMain.run(bankdepositanalysisdbfs,json.dumps( {"FE": [{"transformationsData": {}, "feature": "age", "transformation": "", "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "5647", "mean": "40.94", "stddev": "11.79", "min": "18", "max": "95", "missing": "0"}}, {"transformationsData": {"feature_label": "job"}, "feature": "job", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "5647", "mean": "", "stddev": "", "min": "admin.", "max": "unknown", "missing": "0"}, "transformation": "String Indexer"}, {"transformationsData": {"feature_label": "marital"}, "feature": "marital", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "5647", "mean": "", "stddev": "", "min": "divorced", "max": "single", "missing": "0"}, "transformation": "String Indexer"}, {"transformationsData": {"feature_label": "education"}, "feature": "education", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "5647", "mean": "", "stddev": "", "min": "primary", "max": "unknown", "missing": "0"}, "transformation": "String Indexer"}, {"transformationsData": {"feature_label": "default"}, "feature": "default", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "5647", "mean": "", "stddev": "", "min": "no", "max": "yes", "missing": "0"}, "transformation": "String Indexer"}, {"transformationsData": {}, "feature": "balance", "transformation": "", "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "5647", "mean": "1537.64", "stddev": "3157.01", "min": "-2712", "max": "66653", "missing": "0"}}, {"transformationsData": {"feature_label": "housing"}, "feature": "housing", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "5647", "mean": "", "stddev": "", "min": "no", "max": "yes", "missing": "0"}, "transformation": "String Indexer"}, {"transformationsData": {"feature_label": "loan"}, "feature": "loan", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "5647", "mean": "", "stddev": "", "min": "no", "max": "yes", "missing": "0"}, "transformation": "String Indexer"}, {"transformationsData": {"feature_label": "contact"}, "feature": "contact", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "5647", "mean": "", "stddev": "", "min": "cellular", "max": "unknown", "missing": "0"}, "transformation": "String Indexer"}, {"transformationsData": {}, "feature": "day", "transformation": "", "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "5647", "mean": "15.48", "stddev": "8.43", "min": "1", "max": "31", "missing": "0"}}, {"transformationsData": {"feature_label": "month"}, "feature": "month", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "5647", "mean": "", "stddev": "", "min": "apr", "max": "sep", "missing": "0"}, "transformation": "String Indexer"}, {"transformationsData": {}, "feature": "duration", "transformation": "", "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "5647", "mean": "372.74", "stddev": "342.15", "min": "3", "max": "3253", "missing": "0"}}, {"transformationsData": {}, "feature": "campaign", "transformation": "", "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "5647", "mean": "2.51", "stddev": "2.6", "min": "1", "max": "43", "missing": "0"}}, {"transformationsData": {}, "feature": "pdays", "transformation": "", "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "5647", "mean": "51.92", "stddev": "110.33", "min": "-1", "max": "854", "missing": "0"}}, {"transformationsData": {}, "feature": "previous", "transformation": "", "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "5647", "mean": "0.81", "stddev": "2.17", "min": "0", "max": "41", "missing": "0"}}, {"transformationsData": {"feature_label": "poutcome"}, "feature": "poutcome", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "5647", "mean": "", "stddev": "", "min": "failure", "max": "unknown", "missing": "0"}, "transformation": "String Indexer"}, {"transformationsData": {"feature_label": "deposit"}, "feature": "deposit", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "5647", "mean": "", "stddev": "", "min": "no", "max": "yes", "missing": "0"}, "transformation": "String Indexer"}, {"feature": "job_transform", "transformation": "", "transformationsData": {}, "type": "real", "selected": "True", "stats": {"count": "5647", "mean": "2.81", "stddev": "2.72", "min": "0.0", "max": "11.0", "missing": "0"}}, {"feature": "marital_transform", "transformation": "", "transformationsData": {}, "type": "numeric", "selected": "True", "stats": {"count": "5647", "mean": "0.54", "stddev": "0.69", "min": "0", "max": "2", "missing": "0"}}, {"feature": "education_transform", "transformation": "", "transformationsData": {}, "type": "numeric", "selected": "True", "stats": {"count": "5647", "mean": "0.72", "stddev": "0.85", "min": "0", "max": "3", "missing": "0"}}, {"feature": "default_transform", "transformation": "", "transformationsData": {}, "type": "numeric", "selected": "True", "stats": {"count": "5647", "mean": "0.02", "stddev": "0.13", "min": "0", "max": "1", "missing": "0"}}, {"feature": "housing_transform", "transformation": "", "transformationsData": {}, "type": "numeric", "selected": "True", "stats": {"count": "5647", "mean": "0.48", "stddev": "0.5", "min": "0", "max": "1", "missing": "0"}}, {"feature": "loan_transform", "transformation": "", "transformationsData": {}, "type": "numeric", "selected": "True", "stats": {"count": "5647", "mean": "0.13", "stddev": "0.33", "min": "0", "max": "1", "missing": "0"}}, {"feature": "contact_transform", "transformation": "", "transformationsData": {}, "type": "numeric", "selected": "True", "stats": {"count": "5647", "mean": "0.35", "stddev": "0.61", "min": "0", "max": "2", "missing": "0"}}, {"feature": "month_transform", "transformation": "", "transformationsData": {}, "type": "real", "selected": "True", "stats": {"count": "5647", "mean": "3.0", "stddev": "2.85", "min": "0.0", "max": "11.0", "missing": "0"}}, {"feature": "poutcome_transform", "transformation": "", "transformationsData": {}, "type": "numeric", "selected": "True", "stats": {"count": "5647", "mean": "0.45", "stddev": "0.86", "min": "0", "max": "3", "missing": "0"}}, {"feature": "deposit_transform", "transformation": "", "transformationsData": {}, "type": "numeric", "selected": "True", "stats": {"count": "5647", "mean": "0.47", "stddev": "0.5", "min": "0", "max": "1", "missing": "0"}}]}))

	transformationPostExecutionHook(bankdepositanalysisautofe)

except Exception as ex: 
	logging.error(ex)


**TRAIN MODEL**

In [ ]:
%run bankdepositanalysisHooks.ipynb
try:
	mlPreExecutionHook()

	dataAutoML=functionClassification(bankdepositanalysisautofe, ["age", "job", "marital", "education", "default", "balance", "housing", "loan", "contact", "day", "month", "duration", "campaign", "pdays", "previous", "poutcome"], "deposit")

	mlPostExecutionHook(dataAutoML)

except Exception as ex: 
	logging.error(ex)


**PREDICT ON TRAINED MODEL**

In [ ]:
import pandas as pd
import sklearn.metrics

try:
    model=dataAutoML['model']
    X_test=dataAutoML['X_test']
    y_test=dataAutoML['y_test']
    label=dataAutoML['label']
    columnNames=dataAutoML['columnNames']
    if label in columnNames:
        columnNames.remove(label)
    predicted=label+"_predicted"
    y_predicted=model.predict(X_test)
    df =pd.DataFrame(X_test , columns=columnNames)
    df[label]=y_test
    df[predicted]=y_predicted
    columnNames.insert(0,predicted)
    columnNames.insert(0,label)
    Accuracy = round((100 * sklearn.metrics.accuracy_score(y_true=y_test, y_pred=y_predicted)), 1)
    F1= round(
            (100 * sklearn.metrics.f1_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    Precision= round((
                100 * sklearn.metrics.precision_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    Recall = round((
                100 * sklearn.metrics.recall_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    display(" Accuracy of Prediction on test data    : %s"%Accuracy)
    display(" F1 score of Prediction on test data    : %s"%F1)
    display(" Precision of Prediction on test data   : %s"%Precision)
    display(" Recall of Prediction on test data      : %s"%Recall)
    display(df.head())
except Exception as ex:
    logging.error(ex)

